In [2]:
import pandas as pd
from selenium import webdriver as wb
from bs4 import BeautifulSoup
import requests
from time import sleep
import numpy as np
from tqdm import tqdm

In [2]:
data = pd.read_excel('Discrete Choice Data/Data/Parsing/data_model_V2.xlsx')
data = data[(data.transport_type == 'Автомобильный') & (data.price.isna())]

In [7]:
bus_pairs = list(zip(data.source_tc, data.target_tc))
bus_pairs[0:5]

[('Абакан', 'Ачинск'),
 ('Абакан', 'Ачинск'),
 ('Абакан', 'Кемерово'),
 ('Абакан', 'Кемерово'),
 ('Абакан', 'Копьево')]

In [ ]:
from tqdm import tqdm

data_buses = []
errors = []
for idx, row in tqdm(bus_pairs.iterrows()):
    dep_name = row["departure_city_name"]
    arr_name = row["arrival_city_name"]
    dep_id = row["departure_city_id"]
    arr_id = row["arrival_city_id"]
#     time = []
#     price = []
    link = f"https://bus.tutu.ru/расписание_автобусов/{dep_name}/{arr_name}/?from={dep_id}&to={arr_id}&amount=1"
    url = requests.get(link).text
    soup = BeautifulSoup(url,'lxml')
    
    try:
    
        time =  soup.find('span', {'data-ti': "duration"}).text
        price = soup.find('span', {'class': "index__price___3IUNa"}).text

        row_vals = { 
            'source_tc': dep_name,
            'target_tc': arr_name,
            'time': time,
            'price': price
        }

        data_buses.append(row_vals)

    except: errors.append([row["departure_city_name"], row["arrival_city_name"], 
                           row["departure_city_id"], row["arrival_city_id"]])
    

In [23]:
link = 'https://bus.tutu.ru/расписание_автобусов/Владивосток/Хабаровск/?from=1297864&to=1315652&amount=1'
url = requests.get(link).text
soup = BeautifulSoup(url,'lxml')

In [26]:
soup.find('span', {'class': "index__price___3IUNa"}).text

'от 3188 ₽'

In [ ]:
minutes = []
for time_val in df_buses.time:
    if 'ч' in str(time_val):
        l = time_val.split(' ')
        if len(l) > 2:
            minutes.append((int(l[0]) * 60) + int(l[2]))
        else:
            minutes.append((int(l[0]) * 60))
    else:
        try:
            l = time_val.split(' ')
            minutes.append((int(l[0])))
        except: minutes.append(np.nan)

In [38]:
df_buses['price'] = df_buses['price'].str.split(' ').str[1]
df_buses['minutes'] = minutes
df_buses

,source_tc,target_tc,time,price,minutes
0,Владивосток,Хабаровск,15 ч 29 м,3188,929.0
1,Райчихинск,Благовещенск,2 ч 30 м,573,150.0
2,Томск,Барнаул,9 ч 30 м,1745,570.0
3,Томск,Яровое,12 ч,2215,720.0
4,Томск,Красноярск,10 ч 45 м,1566,645.0
...,...,...,...,...,...
158,Севастополь,Воронеж,28 ч 20 м,4372,1700.0
159,Севастополь,Астрахань,27 ч 55 м,3798,1675.0
160,Севастополь,Москва,31 ч,3526,1860.0
161,Нижневартовск,Уфа,27 ч,4176,1620.0


In [39]:
df_buses.to_excel('tutu_busesV2.xlsx')